# Can we predict the temperature given humidity, pressure and wind speed?

### Load packages and prepare the data.

In [4]:
from matplotlib import pyplot as plt
import numpy as np
import mogreps
import netCDF4
import pandas as pd
import scipy.stats as stats


from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
#Read in the data.
data_set = netCDF4.Dataset('ensemble_means_201401_lat250_lon100.nc')
data_setFeb = netCDF4.Dataset('ensemble_means_201402_lat250_lon100.nc')
data_setMar = netCDF4.Dataset('ensemble_means_201403_lat250_lon100.nc')
data_settestJan = netCDF4.Dataset('test_data_201401_lat250_lon100.nc')
data_settestFeb = netCDF4.Dataset('test_data_201402_lat250_lon100.nc')
data_settestMar = netCDF4.Dataset('test_data_201403_lat250_lon100.nc')

data_set_regional = netCDF4.Dataset('ensemble_regional_means_201401.nc')
data_set_regional_test = netCDF4.Dataset('regional_test_data_201401.nc')
data_set_regional_Feb = netCDF4.Dataset('ensemble_regional_means_201402.nc')
data_set_regional_test_Feb = netCDF4.Dataset('regional_test_data_201402.nc')

FileNotFoundError: [Errno 2] No such file or directory: b'ensemble_regional_means_201401.nc'

In [ ]:
#Prepare the data.

temperatureJan = data_set.variables['Temperature']
pressureJan = data_set.variables['Pressure']
humidityJan = data_set.variables['Humidity']
windspeedJan = data_set.variables['Wind Speed']

temperatureFeb = data_setFeb.variables['Temperature']
pressureFeb = data_setFeb.variables['Pressure']
humidityFeb = data_setFeb.variables['Humidity']
windspeedFeb = data_setFeb.variables['Wind Speed']

temperatureMar = data_setMar.variables['Temperature']
pressureMar = data_setMar.variables['Pressure']
humidityMar = data_setMar.variables['Humidity']
windspeedMar = data_setMar.variables['Wind Speed']

temptestJan = data_settestJan.variables['Temperature']
pressuretestJan = data_settestJan.variables['Pressure']
humiditytestJan = data_settestJan.variables['Humidity']
windspeedtestJan = data_settestJan.variables['Wind Speed']

temptestFeb = data_settestFeb.variables['Temperature']
pressuretestFeb = data_settestFeb.variables['Pressure']
humiditytestFeb = data_settestFeb.variables['Humidity']
windspeedtestFeb = data_settestFeb.variables['Wind Speed']

temptestMar = data_settestMar.variables['Temperature']
pressuretestMar = data_settestMar.variables['Pressure']
humiditytestMar = data_settestMar.variables['Humidity']
windspeedtestMar = data_settestMar.variables['Wind Speed']

temptestregJan = data_set_regional_test.variables['Temperature']
pressuretestregJan = data_set_regional_test.variables['Pressure']
humiditytestregJan = data_set_regional_test.variables['Humidity']
windtestregJan = data_set_regional_test.variables['Wind Speed']


temperatureregJan = data_set_regional.variables['Temperature']
pressureregJan = data_set_regional.variables['Pressure']
humidityregJan = data_set_regional.variables['Humidity']
windregJan = data_set_regional.variables['Wind Speed']

temptestregFeb = data_set_regional_test_Feb.variables['Temperature']
pressuretestregFeb = data_set_regional_test_Feb.variables['Pressure']
humiditytestregFeb = data_set_regional_test_Feb.variables['Humidity']
windtestregFeb = data_set_regional_test_Feb.variables['Wind Speed']


temperatureregFeb = data_set_regional_Feb.variables['Temperature']
pressureregFeb = data_set_regional_Feb.variables['Pressure']
humidityregFeb = data_set_regional_Feb.variables['Humidity']
windregFeb = data_set_regional_Feb.variables['Wind Speed']


In [ ]:
temperature_train = np.concatenate([temperatureJan,temperatureFeb,temperatureMar])
pressure_train = np.concatenate([pressureJan,pressureFeb,pressureMar])
humidity_train = np.concatenate([humidityJan,humidityFeb,humidityMar])
wind_train = np.concatenate([windspeedJan,windspeedFeb,windspeedMar])

temperature_test = np.concatenate([np.mean(temptestJan,axis=1),np.mean(temptestFeb,axis=1),np.mean(temptestMar,axis=1)])
pressure_test = np.concatenate([np.mean(pressuretestJan,axis=1),np.mean(pressuretestFeb,axis=1),np.mean(pressuretestMar,axis=1)])
humidity_test = np.concatenate([np.mean(humiditytestJan,axis=1),np.mean(humiditytestFeb,axis=1),np.mean(humiditytestMar,axis=1)])
wind_test = np.concatenate([np.mean(windspeedtestJan,axis=1),np.mean(windspeedtestFeb,axis=1),np.mean(windspeedtestMar,axis=1)])

In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import neural_network
from sklearn import svm


### Take a look at the data.

In [ ]:
d={'humidity': humidity_train, 'pressure': pressure_train, 'wind speed': wind_train, 'temperature': temperature_train}
df = pd.DataFrame(data=d)
df.describe()

In [ ]:
df.hist(figsize=(8,6))
plt.show()

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(df,figsize=(8,6))
plt.show()

### Linear regression.

In [ ]:
predictors_train=np.stack((humidity_train,pressure_train,wind_train),axis=1)
predictors_test=np.stack((humidity_test,pressure_test,wind_test),axis=1)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(predictors_train,temperature_train)

# Make predictions using the testing set
temperature_pred = regr.predict(predictors_test)

# The coefficients.
print('Coefficients: \n', regr.coef_)
# The mean squared error.
print("Mean squared error: %.2f"
      % mean_squared_error(temperature_test, temperature_pred))
# Explained variance score: 1 is perfect prediction.
print('Variance score: %.2f' % r2_score(temperature_test, temperature_pred))



In [ ]:
plt.plot(temperature_test,label='Test temperature')
plt.plot(temperature_pred,'m',label='Predicted temperature')
plt.legend()
plt.xlabel('Time [days]')
plt.ylabel('Temperature [K]')
plt.show()

In [ ]:
plt.plot(np.sqrt((temperature_test-temperature_pred)**2))
plt.xlabel('Test temperature')
plt.ylabel('Predicted temperature')
#plt.legend()
plt.show()

In [ ]:
plt.plot(temperature_pred - temperature_test,'rx')
plt.show()

# Multi-layer regression.

In [ ]:
#MLPRegressor

# Create multilevel regression object.
regrML = neural_network.MLPRegressor(solver='lbfgs')

# Train the model using the training sets.
regrML.fit(predictors_train,temperature_train)

# Make predictions using the testing set.
temperature_predML = regrML.predict(predictors_test)


In [ ]:
# The coefficients.
# The mean squared error.
print("Mean squared error: %.2f"
      % mean_squared_error(temperature_test, temperature_predML))
# Explained variance score: 1 is perfect prediction.
print('Variance score: %.2f' % r2_score(temperature_test, temperature_predML))

#print(regrML.n_iter_)
print('Number of layers: %.2f' % regrML.n_layers_)

In [ ]:
plt.plot(temperature_test,label='Test temperature')
plt.plot(temperature_pred,'m',label='Predicted temperature')
plt.plot(temperature_predML,'b',label='ML Predicted temperature')
plt.xlabel('Time [days]')
plt.ylabel('Temperature [K]')
plt.legend()
plt.show()

In [ ]:
plt.plot(temperature_pred, temperature_test,'bx',label='Linear')
plt.plot(temperature_predML, temperature_test,'rx',label='Multi-layer')
plt.xlabel('Temperature: predicted')
plt.ylabel('Temperature: empirical')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.sqrt((temperature_test-temperature_pred)**2),label='Linear')
plt.plot(np.sqrt((temperature_test-temperature_predML)**2),'r',label='Multi-layer')
plt.xlabel('Test temperature')
plt.ylabel('Predicted temperature')
plt.legend()
plt.show()

## How does machine learning compare to persistance?

Both machine learning experiments result in quite high RMS values. We wish to see if the skill of either our linear regression or three layer neural network beat persistence. We will use temperatures from the previous day to test this.

In [ ]:
plt.plot(temperature_test[1:89],'b',label='Test data')
plt.plot(temperature_test[0:88],'r',label='Persistence')
plt.legend()
plt.show()

In [ ]:
# Plot to show square root error of each estimate
plt.plot(np.sqrt((temperature_test-temperature_pred)**2),label='Linear regression')
plt.plot(np.sqrt((temperature_test-temperature_predML)**2),'r',label='Multi level')
plt.plot(np.sqrt((temperature_test[1:89]-temperature_test[0:88])**2),'k',label='Persistence')

plt.xlabel('Test temperature')
plt.ylabel('Predicted temperature')
plt.legend()
plt.show()

In [ ]:
#np.sum((((temperature_test[1:89]-temperature_pred[1:89])/(temperature_test[1:89]-temperature_test[0:88]))**2)>1)
np.sum((((temperature_test[1:89]-temperature_pred[1:89])/(temperature_test[1:89]-temperature_test[0:88]))**2)<1)

Linear model beats persistence for 54 out of 89 days.

In [ ]:
#np.sum((((temperature_test[1:89]-temperature_pred[1:89])/(temperature_test[1:89]-temperature_test[0:88]))**2)>1)
np.sum((((temperature_test[1:89]-temperature_predML[1:89])/(temperature_test[1:89]-temperature_test[0:88]))**2)<1)

## Will our predictions improve if we normalise the variables? 



In [ ]:
#functions to normalise and denormalise the variables
def normalize(value, min, max):
    normalized = (value - min) / (max - min);
    return normalized;

def denormalize(normalized, min, max):
    denormalized = (normalized * (max - min) + min);
    return denormalized;

In [ ]:
## Normalise the data sets 

norm_temperature_train = normalize(temperature_train,np.min(temperature_train),np.max(temperature_train))
norm_humidity_train = normalize(humidity_train,np.min(humidity_train),np.max(humidity_train))
norm_pressure_train = normalize(pressure_train,np.min(pressure_train),np.max(pressure_train))
norm_wind_train = normalize(wind_train,np.min(wind_train),np.max(wind_train))

norm_temperature_test = normalize(temperature_test,np.min(temperature_test),np.max(temperature_test))
norm_humidity_test = normalize(humidity_test,np.min(humidity_test),np.max(humidity_test))
norm_pressure_test = normalize(pressure_test,np.min(pressure_test),np.max(pressure_test))
norm_wind_test = normalize(wind_test,np.min(wind_test),np.max(wind_test))

#print(min(norm_temperature_train))

In [ ]:
# Split the targets into training/testing sets.


norm_predictors_train=np.stack((norm_humidity_train,norm_pressure_train,norm_wind_train),axis=1)
norm_predictors_test=np.stack((norm_humidity_test,norm_pressure_test,norm_wind_test),axis=1)

# Create linear regression object
norm_regr = linear_model.LinearRegression()

# Train the model using the training sets
norm_regr.fit(norm_predictors_train,norm_temperature_train)

# Make predictions using the testing set
norm_temperature_pred = norm_regr.predict(norm_predictors_test)

# The coefficients.
print('Coefficients: \n', norm_regr.coef_)
# The mean squared error.
print("Mean squared error: %.2f"
      % mean_squared_error(norm_temperature_test, norm_temperature_pred))
# Explained variance score: 1 is perfect prediction.
print('Variance score: %.2f' % r2_score(norm_temperature_test, norm_temperature_pred))


In [ ]:
plt.plot(norm_temperature_test,label='Test temperature')
plt.plot(norm_temperature_pred,'m',label='Predicted temperature')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.sqrt((temperature_test-temperature_pred)**2))
plt.plot(np.sqrt((denormalize(norm_temperature_test,np.min(temperature_test),np.max(temperature_test))
                  -denormalize(norm_temperature_pred,np.min(temperature_test),np.max(temperature_test)))**2),'r')
plt.xlabel('Test temperature')
plt.ylabel('Predicted temperature')
plt.legend()
plt.show()

### No, we get exactly the same output.

# Will our results be better if we use data that is smoothed?

Smoothing should remove some of the noise from our data and may make it easier for our regression/NN to obtain good results

In [ ]:
humidityreg = np.concatenate([humidityregJan,humidityregFeb])
pressurereg = np.concatenate([pressureregJan,pressureregFeb])
temperaturereg = np.concatenate([temperatureregJan,temperatureregFeb])
windreg = np.concatenate([windregJan,windregFeb])

humiditytestreg = np.concatenate([humiditytestregJan,humiditytestregFeb])
pressuretestreg = np.concatenate([pressuretestregJan,pressuretestregFeb])
temperaturetestreg = np.concatenate([temptestregJan,temptestregFeb])
windtestreg = np.concatenate([windtestregJan,windtestregFeb])

In [ ]:
# Split the targets into training/testing sets.
##temperature_train = np.array(traindata['Y'])
#temperature_test = np.array(testdata['Y'])

predictors_train_sm=np.stack((humidityreg,pressurereg,windreg),axis=1)
predictors_test_sm=np.stack((np.mean(humiditytestreg,axis=1),np.mean(pressuretestreg,axis=1),np.mean(windtestreg,axis=1)),axis=1)

# Create linear regression object
regr_sm = linear_model.LinearRegression()

# Train the model using the training sets
regr_sm.fit(predictors_train_sm,temperaturereg)

# Make predictions using the testing set
temperature_pred_sm = regr_sm.predict(predictors_test_sm)

# The coefficients.
print('Coefficients: \n', regr_sm.coef_)
# The mean squared error.
print("Mean squared error: %.2f"
      % mean_squared_error(temperaturereg, temperature_pred_sm))
# Explained variance score: 1 is perfect prediction.
print('Variance score: %.2f' % r2_score(temperaturereg, temperature_pred_sm))


In [ ]:
plt.plot(np.mean(temperaturetestreg,axis=1),'b')
plt.plot(temperature_pred_sm,'r')
plt.show()

In [ ]:
np.corrcoef(np.mean(temperaturetestreg,axis=1),temperature_pred_sm)

In [ ]:
np.corrcoef(temperature_test,temperature_pred)

In [ ]:
meantemptest = np.mean(temperaturetestreg,axis=1)

print(np.corrcoef(meantemptest[0:29],meantemptest[1:30]))

In [ ]:
plt.plot(meantemptest[0:58])
plt.plot(meantemptest[1:59])
plt.show()

In [ ]:
plt.plot(meantemptest[0:58]-meantemptest[1:59])
plt.show()

# Multi level on smoothed data

In [ ]:
#MLPRegressor

# Create multilevel regression object
regrML_sm = neural_network.MLPRegressor(solver='lbfgs')

# Train the model using the training sets
regrML_sm.fit(predictors_train_sm,temperaturereg)

# Make predictions using the testing set
temperature_predML_sm = regrML_sm.predict(predictors_test_sm)

In [ ]:
# The coefficients.
#print('Coefficients: \n', regrML.coefs_)
# The mean squared error.
print("Mean squared error: %.2f"
      % mean_squared_error(temperaturereg, temperature_predML_sm))
# Explained variance score: 1 is perfect prediction.
print('Variance score: %.2f' % r2_score(temperaturereg, temperature_predML_sm))

#print(regrML.intercepts_)
print(regrML_sm.n_iter_)
#intercepts_
print('Number of layers: %.2f' % regrML_sm.n_layers_)

In [ ]:
plt.plot(temperaturereg,'r',label='Test temperature')
plt.plot(temperature_pred_sm,'m',label='Predicted temperature')
plt.plot(temperature_predML_sm,'b',label='ML Predicted temperature')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(temperaturereg,temperature_pred_sm))
rms

In [ ]:
rmsML = sqrt(mean_squared_error(temperaturereg,temperature_predML_sm))
rmsML

In [ ]:
rmsper = sqrt(mean_squared_error(temperaturereg[0:58],temperaturereg[1:59]))
rmsper

# Smoothed data and a different time

In [ ]:
data_set_regional_t3 = netCDF4.Dataset('ensemble_regional_means_201401_t3.nc')
data_set_regional_test_t3 = netCDF4.Dataset('regional_test_data_201401_t3.nc')

temptestreg_t3 = data_set_regional_test_t3.variables['Temperature']
pressuretestreg_t3 = data_set_regional_test_t3.variables['Pressure']
humiditytestreg_t3 = data_set_regional_test_t3.variables['Humidity']
windtestreg_t3 = data_set_regional_test_t3.variables['Wind Speed']


temperaturereg_t3 = data_set_regional_t3.variables['Temperature']
pressurereg_t3 = data_set_regional_t3.variables['Pressure']
humidityreg_t3 = data_set_regional_t3.variables['Humidity']
windreg_t3 = data_set_regional_t3.variables['Wind Speed']

In [ ]:
#MLPRegressor
predictors_train_sm_t3=np.stack((humidityreg_t3,pressurereg_t3,windreg_t3),axis=1)
predictors_test_sm_t3=np.stack((np.mean(humiditytestreg_t3,axis=1),np.mean(pressuretestreg_t3,axis=1),np.mean(windtestreg_t3,axis=1)),axis=1)


# Create linear regression object
regr_sm_t3 = linear_model.LinearRegression()

# Train the model using the training sets
regr_sm_t3.fit(predictors_train_sm_t3,temperaturereg_t3)

# Make predictions using the testing set
temperature_pred_sm_t3 = regr_sm_t3.predict(predictors_test_sm_t3)

# The coefficients.
print('Coefficients: \n', regr_sm_t3.coef_)
# The mean squared error.
print("Mean squared error: %.2f"
      % mean_squared_error(temperaturereg_t3, temperature_pred_sm_t3))
# Explained variance score: 1 is perfect prediction.
print('Variance score: %.2f' % r2_score(temperaturereg_t3, temperature_pred_sm_t3))


In [ ]:


# Create multilevel regression object
regrML_sm_t3 = neural_network.MLPRegressor(solver='lbfgs')

# Train the model using the training sets
regrML_sm_t3.fit(predictors_train_sm_t3,temperaturereg_t3)

# Make predictions using the testing set
temperature_predML_sm_t3 = regrML_sm_t3.predict(predictors_test_sm_t3)

In [ ]:
# The coefficients.
#print('Coefficients: \n', regrML.coefs_)
# The mean squared error.
print("Mean squared error: %.2f"
      % mean_squared_error(temperaturereg_t3, temperature_predML_sm_t3))
# Explained variance score: 1 is perfect prediction.
print('Variance score: %.2f' % r2_score(temperaturereg_t3, temperature_predML_sm_t3))

#print(regrML.intercepts_)
print(regrML_sm_t3.n_iter_)
#intercepts_
print('Number of layers: %.2f' % regrML_sm_t3.n_layers_)

In [ ]:
rmsML_t3 = sqrt(mean_squared_error(temperaturereg_t3,temperature_predML_sm_t3))
rmsML_t3